In [3]:
import pandas as pd
import numpy as np
import sklearn

In [2]:
file_path = 'data/cosmicclassifierTraining.csv'
df = pd.read_csv(file_path)
df.head(5)

,Atmospheric Density,Surface Temperature,Gravity,Water Content,Mineral Abundance,Orbital Period,Proximity to Star,Magnetic Field Strength,Radiation Levels,Atmospheric Composition Index,Prediction
0,0.472806,NaN,-0.313872,-2.089299,-0.152201,-0.885649,0.900105,NaN,Category_6,0.692907,5.0
1,4.180154,-1.157515,2.430956,-1.595850,-3.188678,-0.609434,-0.199828,Category_9,Category_9,NaN,0.0
2,-0.129008,1.621592,-0.785741,2.081196,-1.413796,-0.095152,-3.502577,NaN,Category_8,-0.677182,4.0
3,-3.122000,-2.299818,1.072092,0.353524,-0.192529,2.917067,-1.972329,NaN,Category_11,0.109429,1.0
4,-1.459426,2.890268,0.148757,-0.804439,0.494875,0.044910,-0.438796,Category_6,Category_10,0.407941,9.0


In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def fill_missing_with_similar(df):
    df_filled = df.copy()
    
    # Select only numeric columns for similarity computation
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    # Drop rows where all numerical values are NaN
    valid_rows = df[numeric_cols].dropna(how='all')
    
    for index, row in df.iterrows():
        if row.isnull().any():
            non_null_mask = row.notnull()
            available_values = row[non_null_mask].values.reshape(1, -1)

            # Ensure valid_rows does not include the current row
            valid_rows_excluding_current = valid_rows.drop(index, errors="ignore")

            if valid_rows_excluding_current.empty:
                continue  # Skip if no valid rows exist

            # Compute similarity with other valid rows
            similarities = cosine_similarity(valid_rows_excluding_current.loc[:, non_null_mask], available_values)
            
            # Find the best-matching row index
            best_match_idx = valid_rows_excluding_current.index[np.argmax(similarities)]

            # Fill missing values from the best-matching row
            df_filled.loc[index] = df_filled.loc[index].fillna(df_filled.loc[best_match_idx])

    return df_filled

# Example usage:
df = fill_missing_with_similar(df)
print(df.head())


ValueError: Input contains NaN.